# 크롤링

##### 시간이 상당히 오래걸립니다.

- https://news.daum.net/breakingnews/economic?regDate=20200617
- 위 링크와 같이 , 경제 뉴스데이터를 활용
- 링크 마지막부분에서 볼 수 있듯이 날짜또한 설정 가능
- 1페이지당 15개의 뉴스가 있고 , 총 20페이지씩 크롤링
- 즉 하루에 300개의 뉴스기사 제목만 활용

### 라이브러리

In [2]:
import konlpy
import pandas as pd
import numpy as np
import urllib.request
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


##### 일별 뉴스기사 url과 content 담기 위한 데이터프레임 생성

In [ ]:
news_df = pd.DataFrame(columns = ['date','url','content'])
news_df

##### 일별  하루당 300개의 뉴스기사 url 데이터  불러오기
- 사용한 뉴스의 기간 : 2017/01/01 ~ 2020/06/17

In [ ]:
import urllib.request
from bs4 import BeautifulSoup

url_1='https://media.daum.net/breakingnews/economic?regDate='
#===============불러올 날짜 범위 설정

import datetime

#2017/01/01 ~ 2020/06/17 까지의 뉴스데이터 일별 url 로드

start_date = datetime.date(2017, 1, 1)
end_date   = datetime.date(2020, 6, 18) 
date_range = [(start_date + datetime.timedelta(n)).strftime('%Y%m%d') \
              for n in range(int ((end_date - start_date).days))]

#==============페이지범위 1~20페이지까지
url_2='&page='
page= range(1,20)
for i in tqdm(date_range):    
    date_url = []
    for j in page:
        mainurl=url_1 +str(i)+ url_2 + str(j) 
        source_code_from_URL = urllib.request.urlopen(mainurl)
        soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
        list_news = soup.find('ul', attrs={'class':'list_news2'})
        list_news_li = list_news.find_all('li')
        for item in list_news_li:
            
            try: #가끔 한두개의 기사에서 에러가 발생하였음 . 
                 #많지않은 데이터의 차이라서, 그냥 버리고자 예외문 사용
                link_txt = item.find('a', attrs={'class':'link_txt'})
                date_url.append(link_txt.get('href'))
            except:
                pass
    news_df = news_df.append({'date' : i, 'url' : date_url}, ignore_index=True)
news_df['date'] = pd.to_datetime(news_df['date'])



In [ ]:
news_df.head()

##### 일별 url을 활용하여 뉴스기사 제목만 크롤링 하기

In [ ]:
import konlpy
from newspaper import Article
from konlpy.tag import Okt
import re
import time
start_time = time.time()
okt=Okt()
index = 0

for date in tqdm(news_df['date']):
    day_content= ''    
    for url in news_df.loc[news_df[news_df['date']==date].index[0],'url']:
        try:
            #기사데이터 불러오기
            article=Article(url, language='ko')
            article.download()
            article.parse()

            #뉴스 기사 제목만 받아오는 과정, 
            news_title = article.title
            
            #정규표현식을 이용한 뉴스기사 정제
            news_title= re.sub('[^가-힣0-9,]', repl= ' ',string =news_title)
            x = ''.join(news_title)
            day_content += x
        except:
            pass
    news_df.loc[index, 'content'] = day_content
    index = index + 1

##### csv파일 생성    

In [ ]:
news_df.to_csv('뉴스데이터20200617.csv')
news_df.head()